In [1]:
import datetime

import numpy as np
import pandas as pd

import vectorbt as vbt

In [2]:
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=7)

interval = "1m"

In [3]:
symbols = [
    "BTC-USD",
    "ETH-USD",
    "ETH-BTC",
]

In [4]:
price = vbt.YFData.download(
    symbols = symbols,
    missing_index='drop',
    interval=interval,
    start = start_time,
    end = end_time
).get('Close')

/home/oskar/anaconda3/envs/vectorbt/lib/python3.10/site-packages/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)


In [5]:
# price

In [6]:
# this is a wierd strategy, but it shows what can be done with custom indicators
def custom_indicator(close, rsi_window = 14, ma_window = 50, entry_rsi = 45, exit_rsi = 55):
    assert isinstance(close, pd.DataFrame) or isinstance(close, pd.Series)

    close_5m = close.resample('5T').last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    
    rsi, _ = rsi.align(
        close,
        broadcast_axis=0,
        method="ffill",
        join='right'
    )

    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()

    # when rsi is above 70 then -1 (sell)
    trend = np.where(rsi > exit_rsi, -1, 0)
    # when rsi is below 30 and price is below moving average then 1 (buy)
    trend = np.where((rsi < entry_rsi) & (close < ma), 1, trend)

    return trend

In [7]:
indicator = vbt.IndicatorFactory(
    class_name = 'Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window', 'entry_rsi', 'exit_rsi'],
    output_names=['value']
).from_apply_func(
    custom_indicator,
    rsi_window=14,
    ma_window=50,
    entry_rsi=45,
    exit_rsi=55,
    keep_pd=True # close will be passed as pandas series instead of numpy array
)

In [8]:
rsi_windows = np.arange(start=10, stop=200, step=15, dtype=int)
ma_windows = np.arange(start=10, stop=200, step=15, dtype=int)
rsi_entries = np.arange(start=10, stop=40, step=10, dtype=float)
rsi_exits = np.arange(start=60, stop=90, step=10, dtype=float)

res = indicator.run(
    price,
    rsi_window=rsi_windows,
    ma_window=ma_windows,
    entry_rsi=rsi_entries,
    exit_rsi=rsi_exits,
    param_product=True
)

In [9]:
# res.value

In [10]:
entries = res.value == 1
exits = res.value == -1
pf = vbt.Portfolio.from_signals(price, entries, exits, fees=0.001, freq=interval)

In [11]:
returns = pf.total_return()

best_params = returns.idxmax()
worst_params = returns.idxmin()

print(f'Best params: {best_params} with return {returns.max()}')
print(f'Worst params: {worst_params} with return {returns.min()}')

Best params: (40, 10, 30.0, 70.0, 'ETH-BTC') with return 0.024126165886428196
Worst params: (10, 55, 30.0, 60.0, 'BTC-USD') with return -0.13839615800585492


In [12]:
# returns.groupby(['comb_entry_rsi', 'comb_exit_rsi']).mean()
returns

comb_rsi_window  comb_ma_window  comb_entry_rsi  comb_exit_rsi  symbol 
10               10              10.0            60.0           BTC-USD   -0.041197
                                                                ETH-USD   -0.045401
                                                                ETH-BTC   -0.009632
                                                 70.0           BTC-USD   -0.037049
                                                                ETH-USD   -0.037319
                                                                             ...   
190              190             30.0            70.0           ETH-USD    0.000000
                                                                ETH-BTC    0.000000
                                                 80.0           BTC-USD    0.000000
                                                                ETH-USD    0.000000
                                                                ETH-BTC    0.000000
Name

In [13]:
# filter returns to only keep one symbol
# returns[returns.index.isin(['BTC-USD'], level='symbol')]

In [20]:
fig = returns.vbt.heatmap(
    x_level='comb_rsi_window',
    y_level='comb_ma_window', 
    slider_level='symbol', symmetric=False,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

In [22]:
fig = returns.vbt.volume(
    x_level='comb_rsi_window',
    y_level='comb_ma_window', 
    z_level='comb_entry_rsi',
    slider_level='symbol',
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

In [17]:
pf.plot(column=worst_params, show_titles=True, title=f'Portfolio {worst_params}')

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '772a9551-cefb-4502-bd65-6ec4186edf64',
              'x': array([datetime.datetime(2023, 5, 20, 3, 9, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 20, 3, 10, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 20, 3, 11, tzinfo=datetime.timezone.utc),
                          ..., datetime.datetime(2023, 5, 27, 3, 3, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 27, 3, 4, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 27, 3, 5, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([26857.95898438, 26858.83789062, 26859.93164062, ..., 26750.07226562,
             

In [18]:
pf.plot(column=best_params, show_titles=True, title=f'Portfolio {best_params}')

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'dd330633-59db-42b7-907b-60462e81e5ea',
              'x': array([datetime.datetime(2023, 5, 20, 3, 9, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 20, 3, 10, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 20, 3, 11, tzinfo=datetime.timezone.utc),
                          ..., datetime.datetime(2023, 5, 27, 3, 3, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 27, 3, 4, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 27, 3, 5, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([0.06738614, 0.06738555, 0.06738339, ..., 0.06855786, 0.06856216,
                 